In [ ]:
#Google Drive baglanti
from google.colab import drive
drive.mount('/content/gdrive') 
%cd gdrive/MyDrive/Derin Öğrenme/
%ls

Mounted at /content/gdrive
/content/gdrive/MyDrive/Derin Öğrenme
alexnet_basarili_model2.h5  alexnet_original.ipynb  elma/
alexnet_basarili_model.h5   basarili_model.h5       ResimSiniflama_Gi.ipynb
alexnet_keras_for2.ipynb    best_model.h5           ResimSiniflama.ipynb
alexnet_keras_for.ipynb     datalar2/               Vggnet16_keras.ipynb
alexnet_keras.ipynb         ders/


In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator  # veriçoğaltmayı bu fonk ile yapılır
from keras.models import Sequential  #bir model içinde ard arıdına gelen konvolüsyonlar , poolingler, full connectedları sıralı olarak birbirine bağlayan fonk 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense  #dense=fulllyconnected layer
from keras import backend as K
#keras üzerinden yappıcaz bu örnekte. veri 

#giris olarak verilecek görüntülerin boyutları
img_width, img_height = 224, 224    #girişleri kaça kaç eğitmek istersin

train_data_yolu= 'elma/train'            ## 'data/train'
validation_data_yolu= 'elma/test'        ##'data/validation'   # validasyonu kullanmışız test kullanmamaışız. tam tersi de olabilirdi.
train_ornek_sayisi=2100 
validation_ornek_sayisi=300 
#epochs=50    # öğrenme sayısı
batch_size=10 # her seferinde eğitim için alınacak veri miktarı - kaçar kçar örnek alıcak? mesela 10 örneği 5er 5er mi alıp eğitsin yoksa 1er 1er mi
#train_ornek_sayisi, validation_onrke_sayisi ile batch_size tam bölünmeli   !!!!

if K.image_data_format()== 'channels_first':  #resimlerin ilk kanalını almayı sağla. rgb katmanından ilk katmanı yani
  input_shape= (3, img_width, img_height)
else:
  input_shape= (img_width, img_height, 3)

In [ ]:
#train işlemi için kullanılan verileri cogaltıyor. ImageDataGenerator ile ön işlem yapıyoruz
train_datalar=ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
#zoomlayıp kırpabilisrsin, shear entropi işlemleridir, horizantal eksenine göre flip yaz (istersek vertical ile de flip yapabiliriz.), rescale ile de normalizasyon yapar 0-255 arasını 0-1 arasına çeker
#datalar = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, heigth_shift_range=0.2,
#       shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# istersek rotation ile döndürme, shift ile kaydırma,  fillmode nearest ile de en yakınındaki piksele göre komple doldurma işlemi, 
#test işlemi için görüntüleri rescale boyutlandırma yapıyor
test_datalar =ImageDataGenerator(rescale=1. / 255)

train_generator=train_datalar.flow_from_directory(
    train_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
#binary classification : class_mode='binary'   yazardık
# multiclass (coklu sınıf): class_mode='categorical' 

validation_generator= test_datalar.flow_from_directory(
    validation_data_yolu,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical')
#binary classification : class_mode='binary'   yazardık
# multiclass (coklu sınıf): class_mode='categorical' 

Found 2100 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [ ]:


epochs=50    # öğrenme sayısı

#Alexnet MODEL OLUŞTURMA
model = Sequential()

model.add(Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),input_shape=input_shape, activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(Conv2D(256,(5,5),padding='same',activation='relu'))   #same yerini 2,2 ???
model.add(BatchNormalization(axis=3))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(Conv2D(384,(3,3),padding='same',activation='relu'))  # same değil 1,1
model.add(BatchNormalization(axis=3))

model.add(Conv2D(384,(3,3),padding='same',activation='relu'))  # same değil 1,1
model.add(BatchNormalization(axis=3))

model.add(Conv2D(256,(3,3),padding='same',activation='relu'))  # same değil 1,1
model.add(BatchNormalization(axis=3))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(Flatten())  #düzleştirme faltten katmanı
model.add(Dense(4096, activation='relu'))  #FC=Dense (fully connected layer oluşturuluyor) --giriş sayısı kadar eklemem lazım.  
model.add(Dense(4096, activation='relu'))
model.add(Dense(3,activation='softmax'))  #3 class olduğu için buraya 3 yazdık!! yani kaç çıkışlı olduğu! binary olsaydı 1 yazardık

#MODEL DERLEME
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

#model.summary()


In [ ]:
#from keras.callbacks import ModelCheckpoint
#checkpoint=ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=1,save_best_only=True, mode='min')  #verbose 0 dersek aşağısı kapanır

#modelin eğitimi
history=model.fit_generator(
    train_generator,
    steps_per_epoch= train_ornek_sayisi // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps= validation_ornek_sayisi // batch_size,
    verbose=1,)      #callbacks=[checkpoint]  
#tarin generator ile eğitimi yap, kaç epochla yapcağını, validation datanın nerede olduğunu, hangi dtaaların kullanılacağını , 

print("Train Acc : %.2f%% " % (history.history['accuracy'][-1]*100))
print("Test Acc : %.2f%% " % (history.history['val_accuracy'][-1]*100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
  1/210 [..............................] - ETA: 34:40 - loss: 1.0993 - accuracy: 0.4000

In [ ]:
#modelde weightler rastsal old için basarılı sonuçları korumak için model kaydeder
model.save_weights('alexnet_orginal_50ep_model.h5')

#test = model.evaluate(validation_generator)

#hoca yazmadı ben yazdım: grafikte görmek için
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')